## Local Setup

If you prefer to work locally, see the following instructions for setting up Python in a virtual environment.
You can then ignore the instructions in "Colab Setup".

If you haven't yet, create a [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html) environment using:
```
conda create --name rl_exercises
conda activate rl_exercises
```
Torch recommends installation using conda rather than pip, so run:
```
conda install pytorch cpuonly -c pytorch
```
If you have a CUDA-enabled GPU and would like to use it, visit [the installation page](https://pytorch.org/get-started/locally/) to see the options available for different CUDA versions.
The remaining dependencies can be installed with pip:
```
pip install matplotlib numpy tqdm "gymnasium[classic-control, other]"
```

Even if you are running the Jupyter notebook locally, please run the code cells in **Colab Setup**, because they define some global variables required later.

## Colab Setup

Google Colab provides you with a temporary environment for python programming.
While this conveniently works on any platform and internally handles dependency issues and such, it also requires you to set up the environment from scratch every time.
The "Colab Setup" section below will be part of **every** exercise and contains utility that is needed before getting started.

There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window).
Any changes you make to the Jupyter notebook itself should be saved to your Google Drive.
We also save all recordings and logs in it by default so that you won't lose your work in the event of an instance timeout.
However, you will need to re-mount your Google Drive and re-install packages with every new instance.

In [ ]:
"""Your work will be stored in a folder called `rl_ws23` by default to prevent Colab 
instance timeouts from deleting your edits.
We do this by mounting your google drive on the virtual machine created in this colab 
session. For this, you will likely need to sign in to your Google account and allow
access to your Google Drive files.
"""

from pathlib import Path
try:
    from google.colab import drive
    drive.mount("/content/gdrive")
    COLAB = True
except ImportError:
    COLAB = False

# Create paths in your google drive
if COLAB:
    DATA_ROOT = Path("/content/gdrive/My Drive/rl_ws23")
    DATA_ROOT.mkdir(parents=True, exist_ok=True)

    DATA_ROOT_STR = str(DATA_ROOT)
    %cd "$DATA_ROOT"
else:
    DATA_ROOT = Path.cwd() / "rl_ws23"

# Install python packages
if COLAB:
    %pip install matplotlib numpy tqdm "gymnasium[classic-control, other]" torch

# Exercise 3

In this homework we will be mainly working on Policy gradients (Lecture 5) 
and Natural Policy Gradients (Lecture 6). We are going to implement the 
REINFORCE, the Policy Gradient Theorem and the Natural Gradient algorithms. 

All homeworks are self-contained.
They can be completed in their respective notebooks.
To edit and re-run code, you can therefore simply edit and restart the code cells below.
When you are finished, you will need to submit the notebook as well as all saved figures (see exercises) as a zip file via Ilias.

We start by importing all the necessary python modules and defining some helper functions which you do not need to change.
Still, make sure you are aware of what they do.

In [ ]:
import time
from dataclasses import dataclass
from typing import Sequence

import matplotlib.pyplot as plt
import numpy as np
import tqdm

# Set random seed and output paths
SEED = 3
OUTPUT_FOLDER = DATA_ROOT / "exercise_3" / time.strftime("%Y-%m-%d_%H-%M")
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)


class ProgressBar:
    def __init__(self, num_iterations: int, verbose: bool = True):
        if verbose:  # create a nice little progress bar
            self.scalar_tracker = tqdm.tqdm(
                total=num_iterations,
                desc="Scalars",
                bar_format="{desc}",
                position=0,
                leave=True,
            )
            progress_bar_format = (
                "{desc} {n_fmt:"
                + str(len(str(num_iterations)))
                + "}/{total_fmt}|{bar}|{elapsed}<{remaining}"
            )
            self.progress_bar = tqdm.tqdm(
                total=num_iterations,
                desc="Iteration",
                bar_format=progress_bar_format,
                position=1,
                leave=True,
            )
            self.verbose = True
        else:
            self.verbose = False

    def __call__(self, **kwargs):
        if self.verbose:
            formatted_scalars = {
                key: "{:.3e}".format(value[-1] if isinstance(value, list) else value)
                for key, value in kwargs.items()
            }
            description = "Scalars: " + ", ".join(
                [f"{key}={value}" for key, value in formatted_scalars.items()]
            )
            self.scalar_tracker.set_description(description)
            self.progress_bar.update(1)


def get_reward_contours(
    b_min: float = -10,
    b_max: float = 0,
    n_points: int = 50,
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Compute reward for policies with all combinations of parameters between b_min and b_max
    in steps of n_points. The policy is tested with the mean predicted action and zero variance.
    Since this is result is deterministic, each policy is only tested once.
    :return:
        k1s: np.ndarray [n_points]: values of k1 tested
        k2s: np.ndarray [n_points]: values of k2 tested
        rewards: np.ndarray [n_points, n_points]: reward for each policy
    """
    env = LinEnv()
    policy = LinPolicy()
    k1s = np.linspace(start=b_min, stop=b_max, num=n_points)
    k2s = np.linspace(start=b_min, stop=b_max, num=n_points)
    rewards = np.zeros((n_points, n_points))
    for i, k1 in enumerate(k1s):
        for j, k2 in enumerate(k2s):
            c_params = np.array([k1, k2, 1])
            policy.update_params(c_params)
            rewards[j, i] = PolicyGradientAlgo.test_policy(
                policy=policy,
                env=env,
                n_trials=1,
            )
    rewards = np.clip(rewards, -1500, 1500)
    return k1s, k2s, rewards


def plot_rewards(
    *reward_curves: np.ndarray,
    colors: Sequence[str],
    labels: Sequence[str],
) -> None:
    plt.figure()
    for reward_curve, color, label in zip(reward_curves, colors, labels):
        plt.plot(reward_curve, color=color, label=label)
    plt.title("Policy Performance during Training")
    plt.xlabel("Iterations")
    plt.ylabel("Mean Rewards")
    plt.legend()


def plot_gradients(
    *gradient_curves: np.ndarray,
    colors: Sequence[str],
    labels: Sequence[str],
) -> None:
    plt.figure()
    for gradient_curve, color, label in zip(gradient_curves, colors, labels):
        plt.plot(gradient_curve.mean(axis=-1), color=color, label=label)
    plt.title("Mean Gradient during Training")
    plt.xlabel("Iterations")
    plt.ylabel("Mean of Parameter Gradients")
    plt.legend()


def plot_parameter_contour(
    *parameter_curves: np.ndarray,
    colors: Sequence[str],
    labels: Sequence[str],
) -> None:
    k1s, k2s, rewards = reward_contour_data
    fig, ax = plt.subplots()
    CS = ax.contour(*np.meshgrid(k1s, k2s), rewards)
    ax.clabel(CS, inline=True, fontsize=10)

    for parameter_curve, color, label in zip(parameter_curves, colors, labels):
        plt.plot(
            parameter_curve[:, 0],
            parameter_curve[:, 1],
            color=color,
            alpha=0.5,
            label=label,
        )
        plt.plot(
            parameter_curve[:, 0],
            parameter_curve[:, 1],
            "x",
            color=color,
            alpha=0.5,
        )
        plt.plot(
            parameter_curve[-1, 0],
            parameter_curve[-1, 1],
            "x",
            color="r",
        )
    plt.title("Trajectory Through Parameter Space")
    plt.xlabel("k1")
    plt.ylabel("k2")
    plt.legend(loc="upper left")


@dataclass
class TrainingResults:
    test_rewards: np.ndarray
    grad_estimates: np.ndarray
    parameters: np.ndarray


def plot_training_results(
    *results: TrainingResults,
    colors: Sequence[str],
    labels: Sequence[str],
    prefix: str,
) -> None:
    # plot rewards of policy during training
    plot_rewards(
        *[result.test_rewards for result in results],
        colors=colors,
        labels=labels,
    )
    save_current_figure(prefix + "_rewards")
    plt.show()

    # plot magnitude of gradients
    plot_gradients(
        *[result.grad_estimates for result in results],
        colors=colors,
        labels=labels,
    )
    save_current_figure(prefix + "_grad_estimates")
    plt.show()

    # plot trajectory of parameters through the parameter space
    plot_parameter_contour(
        *[result.parameters for result in results],
        colors=colors,
        labels=labels,
    )
    save_current_figure(prefix + "_contour")
    plt.show()


def save_current_figure(save_name: str) -> None:
    plt.savefig(str(OUTPUT_FOLDER / f"{save_name}.png"))

## Continuous Control with a Linear Controller

We are going to consider a linear dynamical system with a quadratic reward function.
We will use various algorithms to optimize the parameters of a linear controller.
The learning agent does not have any information about the system dynamics and the reward function. 

### Linear System and Quadratic Reward Function

The linear dynamics are described as follows:

\begin{align}
      \boldsymbol{s'} = 
       \boldsymbol{As} + \boldsymbol{Ba},
\end{align}
where $\boldsymbol{s'}$ denotes the state in the next time step and $\boldsymbol{a}$ is the action input to the system. The identites of the system are given as 
\begin{align}
    \boldsymbol{A} = \begin{bmatrix}
                        1 & 0.1\\
                        0 & 0.99
                      \end{bmatrix}, ~~~~~
     \boldsymbol{B} = \begin{bmatrix}
                        0 \\
                        0.1 
                       \end{bmatrix}.
\end{align}
Thus, we have a two dimensional state-space and a one dimensional action space.

The immediate reward function, is given as
\begin{align}
    r(\boldsymbol{s_t}, a_t) = -\boldsymbol{s_t}^T\boldsymbol{M}\boldsymbol{s_t} - a_t^2Q,
\end{align}
resulting in an episode reward
\begin{align}
    R(\tau) = \sum_t r(\boldsymbol{s_t}, a_t)
\end{align}

The code block which describes the linear system and its corresponding quadratic reward function 
is given below.

In [ ]:
S_DIM = 2
A_DIM = 1
HORIZON = 50


class LinEnv:
    _A = np.array([[1, 0.1], [0, 0.99]])
    _B = np.array([0, 0.1])
    _M = np.array([[10, 0], [0, 1]])
    _Q = np.array([1])
    _s_init = np.array([2, 1])

    def __init__(self) -> None:
        self.t = 0
        self._s = np.array([2, 1])
        self.s_max = np.ones(2) * 12
        self.s_min = -np.ones(2) * 12
        self.a_max = 8
        self.a_min = -8

    def reset(self) -> np.ndarray:
        self._s = self._s_init
        self.t = 0
        return self._s.copy()

    def step(self, action: np.ndarray) -> tuple[np.ndarray, float]:
        reward = -self._s.T @ self._M @ self._s - action.T * self._Q * action
        clipped_action = np.clip(action, self.a_min, self.a_max)
        self._s = self._A @ self._s + self._B * clipped_action
        self._s = np.clip(self._s, self.s_min, self.s_max)
        self.t += 1
        return self._s.copy(), reward.item()


linear_env = LinEnv()

### **Linear Controller**
We consider a linear, stochastic controller of the form
\begin{align}
    \pi(a|\boldsymbol{s}) = \mathcal{N}(a|\boldsymbol{Ks}, \sigma^2) =\frac{1}{\sqrt{2\pi \sigma^2}}e^{-\frac{1}{2}\frac{(a-\boldsymbol{Ks})^2}{\sigma^2}},
\end{align}
where $\boldsymbol{K} = [k_1, k_2]$ and $\sigma$ are the learnable parameters.
Our learning algorithms will optimize these parameters.

The following code defines the linear controller

In [ ]:
class LinPolicy:
    _params: np.ndarray
    _K: np.ndarray
    _std: np.ndarray  # standard deviation, i.e. sigma

    def __init__(self):
        self._K = np.zeros(S_DIM)
        self._std = np.ones(A_DIM)
        self.update_params(np.array([-10, -10, 1]))

    @property
    def params(self) -> np.ndarray:
        return self._params

    @property
    def n_params(self) -> int:
        return self._params.shape[0]

    def update_params(self, params: np.ndarray) -> None:
        assert len(params) == S_DIM + A_DIM
        self._K[:] = params[:S_DIM]
        self._std[:] = params[S_DIM:]
        self._params = params

    def get_mean(self, state: np.ndarray) -> np.ndarray:
        # expand dims so that dimension matches that of action
        # dot product removes one dimension
        return np.expand_dims(state @ self._K, axis=-1)

    def sample(self, state: np.ndarray) -> np.ndarray:
        mean = self.get_mean(state)
        action = np.random.normal(mean, np.abs(self._std))
        return action

    def grad_log_pi(self, states: np.ndarray, actions: np.ndarray) -> np.ndarray:
        """Get the gradient of the log probability of the given action(s) in the given state(s).
        :param states: np.ndarray [..., S_DIM], where S_DIM is the state dimension
        :param actions: np.ndarray [..., A_DIM], where A_DIM is the action dimension
        """
        std_inv = 1 / (self._std + 1e-20)
        z = actions - self.get_mean(states)
        grad_K = z * states * (std_inv**2)
        grad_sigma = -std_inv + (z**2) * (std_inv**3)
        return np.concatenate((grad_K, grad_sigma), axis=-1)

## Policy Gradient Algorithms

Next, we will create a simple base class for policy gradient algorithms, which all have the same structure.
This structure is generally:

---

- **Repeat**  For $k=1, 2, \dots$
    - run policy: sample trajectories \{$\tau_i$\}, ${i=1,...,N}$ from $\pi_{\boldsymbol{\theta}}(a|\boldsymbol{s})$
    - Estimate the gradient $\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$ : estimate_grad()
    - Update the parameters: grad_ascent_step()

- **Until convergence**

---

Based on this general structure, in the following we define a base class `PolicyGradientAlgorithm`, which contains shared attributes and functions.
The function `train` implements the main algorithm loop.
In this exercise, we will use various learning rates (`lr`) for various algorithms, with and without baselines.
The parameters `n_iters` and `batch_size` stay the same for the algorithms.
Also note that for this continuous control problem, the discount factor $\gamma$ is omitted since we set it to 1.


In [ ]:
class PolicyGradientAlgo:
    def __init__(
        self,
        env: LinEnv,
        policy: LinPolicy,
        n_iters: int = 150,
        batch_size: int = 25,
        lr: float = 1e-4,
        baseline: bool = False,
    ) -> None:
        self.env = env  # current environment
        self.policy = policy  # current policy object
        self.n_iters = n_iters  # number of total iterations
        self.batch_size = batch_size  # number of traj. samples per iteration
        self.lr = lr  # learning rate
        self.baseline = baseline  # whether to use baseline or not

    def estimate_grad(
        self,
        rewards: np.ndarray,
        states: np.ndarray,
        actions: np.ndarray,
    ) -> np.ndarray:
        raise NotImplementedError

    def train(self) -> TrainingResults:
        """
        This function will perform the main loop of the policy gradient algorithms.
        For plotting and saving purposes, it will return the estimated gradients,
        the test rewards and the parameters of each iteration.
        :return:
            TrainingResults:
                estimated_grads: np.ndarray [n_iters x n_params]
                test_rewards: np.ndarray [n_iters]
                parameters: np.ndarray[n_iters+1 x n_params]
        """
        grads = np.empty((self.n_iters, self.policy.n_params))
        test_rewards = np.empty(self.n_iters)
        rewards = np.empty((self.batch_size, HORIZON))
        states = np.empty((self.batch_size, HORIZON, S_DIM))
        actions = np.empty((self.batch_size, HORIZON, A_DIM))
        parameters = np.empty((self.n_iters + 1, self.policy.n_params))
        parameters[0] = self.policy.params
        progress_bar = ProgressBar(num_iterations=self.n_iters)
        for k in range(self.n_iters):
            for j in range(self.batch_size):
                state = self.env.reset()
                for t in range(HORIZON):
                    states[j, t] = state
                    actions[j, t] = self.policy.sample(state)
                    state, rewards[j, t] = self.env.step(actions[j, t])
            grads[k] = self.estimate_grad(rewards, states, actions)
            parameters[k + 1] = self.grad_ascent_step(grads[k], states, actions)
            self.policy.update_params(parameters[k + 1])
            test_rewards[k] = self.test_policy(policy=self.policy, env=self.env)
            progress_bar(test_reward=test_rewards[k])
        return TrainingResults(test_rewards, grads, parameters)

    def grad_ascent_step(
        self,
        grad_estimates: np.ndarray,
        states: np.ndarray | None = None,
        actions: np.ndarray | None = None,
    ):
        """This function performs the gradient ascent step.
        :param grad_estimates: estimates of the gradients for all parameters:
            np.ndarray [n_params]
        :param states all states of the last iteration:
            np.ndarray [batch_size x HORIZON x S_DIM], where S_DIM is the state dimension
        :param actions all taken actions of the last iteration:
            np.ndarray [batch_size x HORIZON x A_DIM], where A_DIM is the action dimension
        :return: updated policy parameters: np.ndarray [n_params]
        """
        return self.policy.params + self.lr * grad_estimates

    @staticmethod
    def test_policy(policy: LinPolicy, env: LinEnv, n_trials: int = 10) -> float:
        ep_rewards = np.empty((n_trials, HORIZON))
        for i in range(n_trials):
            s = env.reset()
            for t in range(HORIZON):
                a = policy.get_mean(s)
                s, r = env.step(a)
                ep_rewards[i, t] = r
        return np.mean(np.sum(ep_rewards, axis=1)).item()


# evaluate a grid of policy parameters for plotting later
reward_contour_data = get_reward_contours()

# **REINFORCE**
We start with the most basic algorithm, **REINFORCE**. The **pseudocode** is given as

---

- **Repeat**  For $k=1, 2, \dots$
    - run policy: sample trajectories \{$\tau_i$\}, ${i=1,...,N}$ from $\pi_{\boldsymbol{\theta}}(a|\boldsymbol{s})$
    - Estimate the gradient:
$\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta}) \approx \frac{1}{N}\sum_{i} \left(\sum_{t}\nabla_{\boldsymbol{\theta}}\log \pi_{\boldsymbol{\theta}}(a_{i,t}|\boldsymbol{s}_{i,t})\right)\left(\sum_{t}\gamma^tr(\boldsymbol{s}_{i,t}a_{i,t})\right)$
    - Update the parameters:
        - $\boldsymbol{\theta}\leftarrow \boldsymbol{\theta} + \alpha \nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$

- **Until convergence**

---

The following class inherits from the `PolicyGradientAlgorithm` class and only needs to override the function 'estimate_grad' according to the pseudo code given above.
This function estimates the gradient of the return with respect to the parameters of the policy. 

## Task 1: REINFORCE (3 Points)
Your task is to implement the `estimate_grad` function according to the pseudocode shown above (**Don't implement the policy update!**).
Through the object attribute `self.baseline`, we decide if we would like to estimate the gradient with or without a baseline. 
Make sure that both options are working in your code. 
Use the following baseline
\begin{align}
    b = \frac{1}{N}\sum_i\sum_tr(\boldsymbol{s}_{i,t}, a_{i,t})
\end{align}

*Hint: To get the gradient of the log policy for current state $\boldsymbol{s}_{i,t}$ and current action $a_{i,t}$ from the rollout i at time step t, you will need to call `self.policy.grad_log_pi(current_state, current_action)`, where current_state is $\boldsymbol{s}_{i,t}$ and current action is $a_{i,t}$.*

After you completed the implementation, run the cell.
After training, it plots the reward curve, the magnitude of the gradients, and the trajectory of the parameters through the parameter space.
We can now compare the performance of the algorithm with and without a baseline.
Notice that with a baseline, we use a learning rate of $10^{-3}$, whereas without a baseline, we use a learning rate of $10^{-4}$.
What happens to each version of the algorithm for different learning rates?

In [ ]:
class REINFORCE(PolicyGradientAlgo):
    def estimate_grad(
        self,
        rewards: np.ndarray,
        states: np.ndarray,
        actions: np.ndarray,
    ) -> np.ndarray:
        """
        This function returns the gradient estimate of the return.
        :param rewards: all training rewards of the last iteration:
            np.ndarray [batch_size x HORIZON]
        :param states: all states of the last iteration:
            np.ndarray [batch_size x HORIZON x S_DIM], where S_DIM is the state dimension
        :param actions: all taken actions of the last iteration:
            np.ndarray [batch_size x HORIZON x A_DIM], where A_DIM is the action dimension
        :return: grad_estimate: the estimated gradient: np.ndarray [n_params]
        """
        ## TODO ##
        if self.baseline:
            advantages = ...
        else:
            advantages = ...
        grad_estimate = ...

        assert grad_estimate.shape == (3,)
        return grad_estimate


np.random.seed(SEED)
reinforce = REINFORCE(
    env=linear_env,
    policy=LinPolicy(),
    lr=1e-4,
    baseline=False,
)
reinforce_results = reinforce.train()

np.random.seed(SEED)
reinforce_with_baseline = REINFORCE(
    env=linear_env,
    policy=LinPolicy(),
    lr=1e-3,
    baseline=True,
)
reinforce_with_baseline_results = reinforce_with_baseline.train()

plot_training_results(
    reinforce_results,
    reinforce_with_baseline_results,
    colors=["cornflowerblue", "blue"],
    labels=["REINFORCE", "REINFORCE with baseline"],
    prefix="reinforce",
)

# Policy Gradient Theorem

**REINFORCE** suffers from high variance in the gradient estimates.
One way to reduce this high variance is to exploit the temporal structure of the trajectory when estimating returns.
This means using the Monte-Carlo estimate of the return to go from the current state, instead of the total return for the whole trajectory.
The pseudo code is given as

---

- **Repeat**  For $k=1, 2, \dots$
    - run policy: sample trajectories \{$\tau_i$\}, ${i=1,...,N}$ from $\pi_{\boldsymbol{\theta}}(a|\boldsymbol{s})$
    - Estimate the gradient:
$\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta}) \approx \frac{1}{N}\sum_{i} \sum_{t}\nabla_{\boldsymbol{\theta}}\log \pi_{\boldsymbol{\theta}}(a_{i,t}|\boldsymbol{s}_{i,t})\left(\sum_{k=t}\gamma^{k-t}r(\boldsymbol{s}_{i,k}a_{i,k}\right)$
    - Update the parameters:
        - $\boldsymbol{\theta}\leftarrow \boldsymbol{\theta} + \alpha \nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$

- **Until convergence**

---

The following class inherits from the `PolicyGradientAlgorithm` class and only needs to override the function 'estimate_grad' according to the pseudo code given above.
This function estimates the gradient of the return with respect to the parameters of the policy.

## Task 2: Policy Gradient Theorem (3 Points)
Your task is to implement the `estimate_grad` function according to the pseudocode shown above (**Don't implement the policy update!**).
Through the object attribute `self.baseline`, we decide if we would like to estimate the gradient with or without a baseline. 
Make sure that both options are working in your code. 

Use the following **time-dependent** baseline for the return starting from state $s_t$
\begin{align}
    b_t = \frac{1}{N}\sum_i\sum_{k=t}r(\boldsymbol{s}_{i,k}, a_{i,k})
\end{align}

After you completed the implementation, run the **Execute Policy Gradient Theorem** cell. This cell will save three different figures which you will need to submit.

*Hint: To get the gradient of the log policy for current state $\boldsymbol{s}_{i,t}$ and current action $a_{i,t}$ from the rollout i at time step t, you will need to call `self.policy.grad_log_pi(current_state, current_action)`, where current_state is $\boldsymbol{s}_{i,t}$ and current action is $a_{i,t}$.*

After you completed the implementation, run the cell.
How do the performance and stability compare to the REINFORCE algorithm?
What happens to each version of the algorithm for different learning rates?

In [ ]:
class PolicyGradientTheorem(PolicyGradientAlgo):
    def estimate_grad(
        self,
        rewards: np.ndarray,
        states: np.ndarray,
        actions: np.ndarray,
    ) -> np.ndarray:
        """
        This function returns the gradient estimate of the return.
        :param rewards: all training rewards of the last iteration: np.ndarray
                     [batch_size x HORIZON]
        :param states: all states of the last iteration: np.ndarray
                     [batch_size x HORIZON x S_DIM], where S_DIM is the state dimension
        :param actions: all taken actions of the last iteration: np.ndarray
                     [batch_size x HORIZON x A_DIM], A_DIM is the action dimension
        :return: grad_estimate: the estimated gradient: np.ndarray [n_params]
        """
        ## TODO ##
        if self.baseline:
            ...
        grad_estimate = ...

        assert grad_estimate.shape == (3,)
        return grad_estimate


np.random.seed(SEED)
pgt = PolicyGradientTheorem(
    env=linear_env,
    policy=LinPolicy(),
    lr=1e-4,
    baseline=False,
)
pgt_results = pgt.train()

np.random.seed(SEED)
pgt_with_baseline = PolicyGradientTheorem(
    env=linear_env,
    policy=LinPolicy(),
    lr=1e-3,
    baseline=True,
)
pgt_with_baseline_results = pgt_with_baseline.train()

plot_training_results(
    pgt_results,
    pgt_with_baseline_results,
    colors=["olive", "green"],
    labels=["Policy Gradient Theorem", "Policy Gradient Theorem with baseline"],
    prefix="pg_theorem",
)

# **Natural Policy Gradient**

A common approach in policy search is to apply a trust region constraint, where the policy update is bounded.
It has been shown that trust regions highly stabilize the learning process. 

A trust region approach which can be applied to continous control problems with parametric policy distributions is **Natural Policy Gradient**.
Here, the KL-constraint is approximated with the second order Taylor approximation, which results in the **Fisher Information** matrix $F$.

Concretely, we can calculate $F$ as 
\begin{align}
    \boldsymbol{F} = \frac{1}{TN}\sum_i^N\sum_t^T \left[\nabla_{\theta}\log \pi_{\theta}(a_{i,t}|s_{i,t})\nabla_{\theta}\log \pi_{\theta}(a_{i,t}|s_{i,t})^T\right].
\end{align}

The policy's parameter update is then given as
\begin{align}
    \boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \alpha\boldsymbol{F}^{-1}\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta}),
\end{align}
where for the gradient estimate $\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$ standard techniques like the policy gradient theorem is used.

In Natural Policy Gradient, the learning rate parameter $\alpha ~~(\eta^{-1}\text{ in the slides})$ can be solved in closed-form by finding the optimal solution of the dual function to the according Lagrangian (see Task 4). More specifically, $\alpha$ is given as 
\begin{align}
    \alpha = \sqrt{\frac{4\epsilon}{\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})^{T}\boldsymbol{F}^{-1}\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})}},
\end{align}
where $\epsilon$ is the hyperparameter, bounding the expected KL between the new and the old policy.

Since we will use the gradient estimate from the **Policy Gradient Theorem**, the following class inherits from the **PolicyGradientTheorem** class, i.e. you need to have properly solved Task 2 in order to be able to solve this task.


## Task 3: Natural Policy Gradient (3 Points)

Implement the function `get_fisher_information`, which will return the Fisher information matrix given state-action trajectories.
Apply the equations mentioned above.
Implement the function `grad_ascent_step`, which calculates and returns the new parameters of the policy according to the update rule mentioned above.
When implementing the update, please also use the closed-form solution to $\alpha$ to scale the update.

*Note: You will need to have properly solved Task 2 in order to be able to solve this task.*

*Hint: To get the gradient of the log policy for current state $\boldsymbol{s}_{i,t}$ and current action $a_{i,t}$ from the rollout i at time step t, you will need to call `self.policy.grad_log_pi(current_state, current_action)`, where current_state is $\boldsymbol{s}_{i,t}$ and current action is $a_{i,t}$.*

After you completed the implementation, run the cell.
What do you notice?
How does this algorithm respond to different hyperparameters?

In [ ]:
class NaturalPolicyGradient(PolicyGradientTheorem):
    def __init__(
        self,
        *args,
        eps: float = 0.5,
        **kwargs,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.eps = eps

    def get_fisher_information(
        self,
        states: np.ndarray,
        actions: np.ndarray,
    ) -> np.ndarray:
        """
        This function calculates the Fisher Information matrix.
        :param states: all states of the last iteration: np.ndarray
                     [batch_size x HORIZON x S_DIM], where S_DIM is the state dimension
        :param actions: all taken actions of the last iteration: np.ndarray
                     [batch_size x HORIZON x A_DIM], A_DIM is the action dimension
        :return: F: returns the Fisher information matrix: np.ndarray [n_params x n_params]
        """
        ## TODO ##
        F = ...

        assert F.shape == (3, 3)
        return F

    def grad_ascent_step(
        self,
        grad_estimates: np.ndarray,
        states: np.ndarray,
        actions: np.ndarray,
    ) -> np.ndarray:
        """
        Performs an updated on the parameters of the policy according to the Natural Policy Gradient Rule by using
        the current gradient estimate (grad_estimate) of the Policy Gradient theorem with baseline, the
        state trajectories and the action trajectories.
        :param grad_estimates: estimates of the gradients for all parameters:
            np.ndarray [n_params]
        :param states all states of the last iteration:
            np.ndarray [batch_size x HORIZON x S_DIM], where S_DIM is the state dimension
        :param actions all taken actions of the last iteration:
            np.ndarray [batch_size x HORIZON x A_DIM], where A_DIM is the action dimension
        :return: updated policy parameters: np.ndarray [n_params]
        """
        ## TODO ##
        new_policy_params = ...

        assert new_policy_params.shape == (3,)
        return new_policy_params


np.random.seed(SEED)
npg = NaturalPolicyGradient(
    env=linear_env,
    policy=LinPolicy(),
    eps=0.5,
    baseline=True,
)
npg_results = npg.train()
plot_training_results(
    npg_results,
    colors=["orange"],
    labels=["Natural Policy Gradient"],
    prefix="npg",
)

The following cell will plot the results from all algorithms into one plot.

In [ ]:
# plot all algorithms in one plot
plot_training_results(
    reinforce_results,
    reinforce_with_baseline_results,
    pgt_results,
    pgt_with_baseline_results,
    npg_results,
    colors=["cornflowerblue", "blue", "olive", "green", "orange"],
    labels=[
        "REINFORCE",
        "REINFORCE with baseline",
        "Policy Gradient Theorem",
        "Policy Gradient Theorem with baseline",
        "Natural Policy Gradient",
    ],
    prefix="all",
)

## **Task 4: Natural Policy Gradient Step Size** (4 Points)

Recall that Natural Gradients use a Taylor approximation of the trust region problem, i.e., the objective is given as 
$$\boldsymbol{g}^* = \underset{\boldsymbol{g}}{\textrm{argmax}} ~~ \boldsymbol{g}^T\nabla_{\boldsymbol{\theta}} \boldsymbol{J} ~~ \textrm{s.t.} ~~ \boldsymbol{g}^T \boldsymbol{F} \boldsymbol{g} \leq \epsilon.$$
By introducing a Lagrangian multiplier $\eta$ we can construct the corresponding Lagrangian
$$ L(\boldsymbol{g}, \eta) = \boldsymbol{g}^T \nabla_{\boldsymbol{\theta}} \boldsymbol{J} + \eta \left(\epsilon - \boldsymbol{g}^T\boldsymbol{F}\boldsymbol{g}\right).$$

.## TODO ##

Derive $\boldsymbol{g}^*$. Also solve the dual, your solution should not depend on $\eta$ anymore!

You may submit your answer in LaTeX as part of this document, or as a separate document included in the same zip file as your submission.
This separate document may be a pdf (e.g. created using word/LaTeX), or a scan of your solution written neatly and legible on a sheet of paper.




## **Discrete Control with Deep Neural Net Controller**

Next, we will consider training Deep Neural Net policies to solve the discrete pole balancing environment called 'CartPole'. We will use the ['CartPole-v1' environment from Gymnasium](https://gymnasium.farama.org/environments/classic_control/cart_pole/).

![The CartPole Environment](https://gymnasium.farama.org/_images/cart_pole.gif)

**Policy gradients With Neural Network Policies.**

We will use the **REINFORCE** algorithm **with baselines** and **policy gradient theorem** as shown below. 

---

- **Repeat**  For $k=1, 2, \dots$
    - run policy: sample trajectories \{$\tau_i$\}, ${i=1,...,N}$ from $\pi_{\boldsymbol{\theta}}(a|\boldsymbol{s})$
    - Estimate the gradient:
$\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta}) \approx \frac{1}{N}\sum_{i} \sum_{t}\nabla_{\boldsymbol{\theta}}\log \pi_{\boldsymbol{\theta}}(a_{i,t}|\boldsymbol{s}_{i,t})\left(Q_t-b_t\right)$
    - Update the parameters:
        - $\boldsymbol{\theta}\leftarrow \boldsymbol{\theta} + \alpha \nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$

- **Until convergence**

---
$Q_t$ is the Q-value at time t, $Q^{\pi}(s_t, a_t)$, and $b_t$ is a baseline.

However we will now replace the linear policies with deep neural networks and compute policy gradients with automatic differentiation. To do this we create a graph in such a way that its gradient is the policy gradient. 

We first import the necessary packages.

In [ ]:
from collections import deque

import gymnasium as gym
import torch
import torch.distributions as distributions
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


def train_once(
    env: gym.Env,
    policy: nn.Module,
    discount: float,
    baseline: bool,
    pg_theorem: bool,
) -> tuple[float, float]:
    policy.train()

    log_prob_actions: list[torch.Tensor] = []
    rewards: list[float] = []

    state, info = env.reset()
    terminated = truncated = False

    while not (terminated or truncated):
        state = torch.from_numpy(state).to(device).unsqueeze(dim=0)

        # get logits from neural network (unnormalized probabilities)
        action_logits = policy(state)

        # create a probability distribution using softmax on the logits
        action_prob = F.softmax(action_logits, dim=-1)

        # Sample discrete actions from a categorical distribution
        # https://pytorch.org/docs/stable/distributions.html
        dist = distributions.Categorical(action_prob)

        # sample an action (without gradients)
        action = dist.sample()

        # Use distribution object to compute log probabilities for the sampled actions.
        # These values have gradients and are used for optimization
        log_prob_action = dist.log_prob(action)

        state, reward, terminated, truncated, info = env.step(
            action.squeeze(dim=0).cpu().numpy()
        )

        log_prob_actions.append(log_prob_action)
        rewards.append(reward)

    returns = calculate_returns(rewards, discount, baseline, pg_theorem)

    loss = update_policy(
        returns=torch.from_numpy(returns).to(device),
        log_prob_actions=torch.cat(log_prob_actions),
    )

    return loss, returns[0].item()


def evaluate(envs: gym.Env, policy: nn.Module) -> float:
    policy.eval()

    return_ = 0

    state, info = envs.reset()
    terminated = truncated = False

    while not (terminated or truncated):
        state = torch.from_numpy(state).to(device).unsqueeze(dim=0)

        with torch.no_grad():
            # get logits from neural network (unnormalized probabilities)
            action_logits = policy(state)

        # create a probability distribution using softmax on the logits
        action_prob = F.softmax(action_logits, dim=-1)

        # select the action with the highest probability
        action = torch.argmax(action_prob, dim=-1)

        state, reward, terminated, truncated, info = envs.step(
            action.squeeze(dim=0).cpu().numpy()
        )

        return_ += reward

    return return_

## Task 5: Building a Neural Network in PyTorch (1 point)
    
This part of the code creates a feed forward neural network using PyTorch, which will form the policy  $\pi_{\boldsymbol{\theta}}(a|\boldsymbol{s})$.

Implement the `__init__` method of the `MLP` class, which creates the neural network as a sequence of alternating `torch.nn.Linear` and `torch.nn.ReLU` layers.
After the `__init__` method, `MLP` should have a `network` attribute which is a `torch.nn.Sequential` object.
The neural network should have `input_size` input nodes, then hidden layers with sizes dictated by `hidden_sizes`, and finally `output_size` output nodes.

*Hint: Take a look at the solutions to Exercise 2 for an example.*

In [ ]:
class MLP(nn.Module):
    def __init__(
        self,
        input_size: int,
        output_size: int,
        hidden_sizes: list[int],
    ) -> None:
        """
        :param state_dim: dimension of the state space
        :param state_dim: dimension of the actions
        :param hidden_units: list of integers corresponding to hidden units
        """
        super().__init__()

        ## TODO ##
        self.network = ...

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        """forward pass of decoder
        :param input:
        :return: output mean
        """
        return self.network(input)

### **Computing Q-values**
The code block computes numpy arrays for Q-values which will be used to compute advantages.

Recall that the expression for the policy gradient "loss" is

\begin{align}
    J = \text{E}_{\tau\sim p(\tau)}\left[\sum_{t=0}^T\nabla_{\boldsymbol{\theta}}\log \pi(a_t|s_t)(Q_t-b_t)\right],
\end{align}
where $ \tau=(s_0, a_0, ...)$ is a trajectory, $Q_t$ is the Q-value at time $t$ and $b_t$ is a baseline.


We can obtain four different cases, depending on whether we use returns to returns to go and whether we subtract a baseline or not:

**Case 1: REINFORCE (pg_theorem = False)**

Here we estimate $Q_t$ by the total discounted return for the entire trajectory, regardless of which time step the Q-value should be for. Therefor, the Q estimator is

\begin{align}
    Q_t = \text{R}(\tau) = \sum_{k=0}^T \gamma^{k} r_{k},
\end{align}

**Case 2: Policy Gradient Theorem (pg_theorem = True)**

Here, we subtract out any rewards from the past, since they cannot have resulted from the current action. Thus, the Q estimator is

\begin{align}
    Q_t = \text{R}(\tau_{k \gt t}) = \sum_{k=t}^T \gamma^{(k-t)} r_{k}
\end{align}

**Case 3: No Baseline (baseline = False)**

Here we use the absolute $Q_t$ as is, with the baseline set to 0:

\begin{align}
    b_t = 0
\end{align}

**Case 4: With baseline (baseline = True)**

We can subtract any baseline from the $Q_t$ as long as it doesn't depend on the trajectory. In this example, since we only have one environment, we take the mean of the $1000$ most recent $Q$ values.

\begin{align}
    b_t = \frac{1}{N}\sum_i^{N} Q_t
\end{align}

In [ ]:
def calculate_returns(
    rewards: list[float],
    discount: float,
    baseline: bool,
    pg_theorem: bool,
) -> np.ndarray:
    returns = []
    return_ = 0
    for reward in reversed(rewards):
        return_ = reward + return_ * discount
        returns.append(return_)
    returns.reverse()

    if not pg_theorem:
        returns = returns[:1] * len(returns)

    returns = np.array(returns)

    if baseline:
        recent_returns.extend(returns)
        mean_return = np.mean(recent_returns)
        returns = returns - mean_return

    return returns

## Task 6: Computing Loss (1 Points)

Here we create a "pseudoloss" which is the weighted maximum likelihood, $\sum_{t=0}^T \log \pi_{\theta}(a_t|s_t)  (Q_t - b_t )$, using the stored `log_prob_actions` and `returns` values computed in the previous section.
The gradient of this loss function with respect to the neural network parameters ($\theta$) is the policy gradient.

In [ ]:
def update_policy(
    returns: torch.Tensor,
    log_prob_actions: torch.Tensor,
) -> float:
    """
    This function calculates the loss and backpropagate the errors using pytorch optmizers.
    :param returns: rewards received from the previous trajectory
        torch.Tensor [HORIZON]
    :param log_prob_actions: log probability of each action from the previous trajectory
        torch.Tensor [HORIZON]
    :param optimizer: the torch optimizer object https://pytorch.org/docs/stable/optim.html
    :return: magnitude of calculated loss (float)
    """

    ## TODO ##
    loss = ...

    ## Backpropagate using pytorch autograd. We will use the Adam Optimizer to do this, though any optimizer
    ## would work in practice.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

## Hyperparameters, run training, and plot reward curve

We run the algorithm 5 times with different random seeds, and plot the mean reward curve with min/max bounds.

You can play around with the hyperparameters and see how each of these 4 affect the algorithms performance. However, please submit the saved figures with the default parameters given here.

In [ ]:
rl_env = gym.make("CartPole-v1")

# Hyperparameters
# learning
baseline = True
pg_theorem = True
learning_rate = 5e-3
discount = 0.99

# network architecture
input_dim = rl_env.observation_space.shape[0]
output_dim = rl_env.action_space.n
hidden_sizes = [32, 16]

# training length and accelerator
N_RUNS = 5
N_ITERATIONS = 200
device = torch.device("cpu")
train_returns = np.zeros((N_RUNS, N_ITERATIONS))
test_returns = np.zeros((N_RUNS, N_ITERATIONS))

# seeding
rl_env.reset(seed=SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


for run in range(N_RUNS):
    policy = MLP(input_dim, output_dim, hidden_sizes).to(device)
    optimizer = optim.Adam(policy.parameters(), lr=learning_rate)
    recent_returns = deque(maxlen=1000)

    progress_bar = ProgressBar(num_iterations=N_ITERATIONS)
    for episode in range(N_ITERATIONS):
        loss, train_reward = train_once(rl_env, policy, discount, baseline, pg_theorem)

        test_reward = evaluate(rl_env, policy)

        train_returns[run][episode] = train_reward
        test_returns[run][episode] = test_reward

        progress_bar(Run=run, train_reward=train_reward, test_reward=test_reward)


# plot the learning curve
plt.figure()
plt.plot(test_returns.mean(axis=0))
plt.fill_between(
    np.arange(N_ITERATIONS),
    test_returns.min(axis=0),
    test_returns.max(axis=0),
    alpha=0.1,
)
plt.xlabel("Training Steps")
plt.ylabel("Mean Eval Reward (Min/Max)")
plt.title("Mean, Min, and Max Eval Reward During Training")
save_current_figure("Deep_PG_Reward")
plt.show()